In [1]:
# imports

import os
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import requests

In [2]:
requests.get("http://localhost:11434").content

b'Ollama is running'

In [3]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

In [4]:
# constants
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')
MODEL_LLAMA = 'llama3.2'

In [5]:
system_promt= """ 
                You are a technical explainer and problem-solver.
Your job is to answer the user’s technical question with a clear, accurate explanation that matches their skill level.

Guidelines:
- Start with a short direct answer.
- Then explain the concept step-by-step in plain language.
- Use a small example when helpful (code, command, or pseudo-code).
- Call out assumptions and edge cases.
- If the question is ambiguous, ask 1–2 clarifying questions, otherwise make reasonable assumptions and state them.
- Be honest about uncertainty; do not invent facts.
- Prefer practical guidance over theory unless the user asks for theory.
Output format:
1) Direct answer
2) Explanation
3) Example (if useful)
4) Common pitfalls / gotchas
5) Next steps (optional)

Respond in markdown with proper label and sub label without code blocks.

"""

In [6]:
# here is the question; type over this to ask something new
def get_question_user_prompt(question):
    question_user_prompt = """ Please explain what this code does and why:
                                   \n """
    question_user_prompt +=question 
    return question_user_prompt
  
#           

In [7]:
question=' yield from {book.get("author") for book in books if book.get("author")} '
get_question_user_prompt(question)

' Please explain what this code does and why:\n                                   \n  yield from {book.get("author") for book in books if book.get("author")} '

In [8]:
def stream_code_explainer_ollama(question):
    stream = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": system_promt},
            {"role": "user", "content": get_question_user_prompt(question)}
          ],
        stream=True  # streanms one by one in chunks (parts)
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
# Get Llama 3.2 to answer
stream_code_explainer_ollama(question)

### Direct Answer
This line of code takes the links from a webpage, retrieves them using the `fetch_website_links` function, and appends these links to a string called `user_prompt`.

### Explanation
Let's break this down step by step:

#### Step 1: Fetching Website Links
The `fetch_website_links(url)` function is used to extract the links from a given URL. This could be done using web scraping techniques.

#### Step 2: Retrieving the Links
The retrieved links are stored in a list called `links`.

#### Step 3: Appending Links to User Prompt
A string called `user_prompt` is updated by appending the joined links to it. The `.join()` method concatenates all elements in the `links` list into a single string.

Assumptions:

* The `fetch_website_links(url)` function returns a list of tuples where each tuple contains the link and other information like title.
* The `user_prompt` variable is initialized as an empty string.

### Example

```python
# fetch_website_links function is not shown here, but it would look something like this:
def fetch_website_links(url):
    import requests
    # send HTTP request to url and parse the HTML content
    response = requests.get(url)
    links = []
    for line in response.text.split('<a href="'):
        link, title = line.strip().split('">')
        links.append((link, title))
    return links

# sample usage with fetch_website_links and append links to user_prompt
url = "https://www.example.com"
links = fetch_website_links(url)
user_prompt += ". ".join([link for link, _ in links])  # getting just the link part
```

### Common Pitfalls / Gotchas

* Make sure `fetch_website_links(url)` returns a list of strings (the actual link) instead of a different data type.
* If there are multiple links on each page with the same suffix, how do you distinguish between them? This may require more complex web scraping techniques.

In [9]:
question=""""week1 EXERCISE.ipynb"links = fetch_website_links(url)
           user_prompt += "\n".join(links) """
stream_code_explainer_ollama(question)

**Direct Answer**
The code reads links from a website using the `fetch_website_links` function, concatenates them into a single string, and appends them to the `user_prompt` variable.

**Explanation**
Let's break down what's happening in this code:

1. **Fetching links**: The `fetch_website_links` function is called with a URL argument (`url`). This function likely sends an HTTP request to the website, parses its HTML content, and extracts URLs (i.e., links) from it.
2. **Concatenating links**: The returned list of links (`links`) is then converted into a single string using the `.join()` method. The string contains all the link values separated by spaces.
3. **Appending to user prompt**: The resulting concatenated links string (`user_prompt`) is appended (i.e., added) to `user_prompt` itself.

**Assumptions**
We assume that:

* The `fetch_website_links` function is defined elsewhere in the codebase and returns a list of URLs.
* `url` contains a valid website URL.
* `user_prompt` is an existing variable or attribute initialized earlier in the code.

**Common Pitfalls / Gotchas**
Be cautious when dealing with link parsing, as some websites use complex structures or scripts to generate links. Also, ensure that the `fetch_website_links` function respects any rate limits, redirects, or other website-specific constraints.

**Example (Pseudo-Code)**
```
links = fetch_website_links("https://example.com")
user_prompt = ""
if links:
    user_prompt = (
        "Visit these websites:\n"
        + "\n".join(links)
    )
```

### Adding UI using gradio

In [10]:
import gradio as gr
import re

d:\LLM\Projects\LLM-Code-Explainer\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def looks_like_code(s: str) -> bool:
    s = s.strip()
    if not s:
        return False

    patterns = [
        r"```",                          # fenced code blocks
        r"^\s{4,}\S",                    # indented lines (4+ spaces)
        r"\b(def|class|import|from|if|elif|else|for|while|try|except|with|return|print)\b",
        r"[(){}\[\];]",                  # code punctuation
        r"==|!=|<=|>=|:=|\+=|-=|\*=|/=|//=|\*\*",
        r":\s*$",                        # line ending with colon (Python blocks)
        r"#",                            # comment
    ]

    score = sum(bool(re.search(p, s, re.MULTILINE)) for p in patterns)
    if score >= 2:
        likely_code=True
    else:
        likely_code=False

    return likely_code



In [ ]:
def generate_code_explainer_ollama(message,history):
    history=[{"role":h['role'],"content":h['content']}for h in history]
    if looks_like_code(message)==True:
        relative_system_promt=system_promt
        relative_user_prompt=get_question_user_prompt(message)
    else:
        relative_system_promt='You are an polite helpful Technical assistant to help learners'
        relative_user_prompt=message
        
    
    stream = ollama.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[{"role": "system", "content": relative_system_promt}] +history+
          [{"role": "user", "content": relative_user_prompt}],
        stream=True  # streanms one by one in chunks (parts)
    )    
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [13]:
chat=gr.ChatInterface(fn=generate_code_explainer_ollama,title='Code Explainer')
chat.launch(share=True,inbrowser=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://eca2cf54197257aca8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
